In [23]:
#SCRIpT 1 : Simulasi Rotasi Baton, angular velocity constan
import tkinter as tk
import math
from vector import Vector  # Pastikan file vector.py ada di direktori yang sama

# Class Baton (tongkat)
class Baton:
    def __init__(self, canvas, center, length):
        self.canvas = canvas
        self.center = center                # Vector posisi tengah
        self.length = length                # Panjang tongkat
        self.angle_deg = 0                  # Sudut awal (dalam derajat)
        self.angular_velocity = 2           # Kecepatan sudut (derajat/frame)
        self.angular_acceleration = 0       # Percepatan sudut (tetap nol)
        
        # Garis tongkat di canvas
        self.line = canvas.create_line(0, 0, 0, 0, width=8, fill="blue")
        
        # Teks informasi di sudut layar
        self.text = canvas.create_text(10, 10, anchor="nw", text="", font=("Arial", 12), fill="black")

    def update(self):
        # Konversi sudut ke radian
        angle_rad = math.radians(self.angle_deg)

        # Buat arah unit vector berdasarkan sudut
        direction = Vector(math.cos(angle_rad), math.sin(angle_rad))

        # Hitung dua ujung tongkat berdasarkan pusat
        end1 = self.center.added(direction.multed(self.length))
        end2 = self.center.subbed(direction.multed(self.length))

        # Perbarui garis tongkat pada canvas
        self.canvas.coords(self.line, end1.x, end1.y, end2.x, end2.y)

        # Tampilkan informasi perhitungan pada layar
        info = f"""Sudut (derajat): {self.angle_deg:.1f}°
Sudut (radian): {angle_rad:.2f}
Angular Velocity: {self.angular_velocity:.2f}°/frame
Angular Acceleration: {self.angular_acceleration:.2f}°/frame²
Ujung 1: ({end1.x:.1f}, {end1.y:.1f})
Ujung 2: ({end2.x:.1f}, {end2.y:.1f})
"""
        self.canvas.itemconfig(self.text, text=info)

        # Update sudut dengan angular velocity
        self.angle_deg = (self.angle_deg + self.angular_velocity) % 360


# Inisialisasi Tkinter dan kanvas
root = tk.Tk()
root.title("Simulasi Rotasi Baton - Nature of Code 3.1")

canvas = tk.Canvas(root, width=600, height=400, bg="white")
canvas.pack()

# Titik tengah canvas sebagai Vector
center = Vector(300, 200)

# Buat objek Baton
baton = Baton(canvas, center, length=100)

# Fungsi animasi
def animate():
    baton.update()
    root.after(50, animate)

animate()
root.mainloop()

In [18]:
import tkinter as tk
import math
from vector import Vector  # Import Vector buatan sendiri

# ==========================
# Class Baton (Tongkat)
# ==========================
class Baton:
    def __init__(self, center, length):
        self.center = center                # Titik pusat rotasi (Vector)
        self.length = length               # Panjang tongkat

        self.angle = 0                     # Sudut awal (radian)
        self.angular_velocity = 0.05       # Kecepatan sudut
        self.angular_acceleration = 0.0005 # Percepatan sudut

        self.line = None                   # ID garis pada canvas
        self.circle1 = None                # Lingkaran ujung kiri
        self.circle2 = None                # Lingkaran ujung kanan

    def update(self):
        # Update sudut dan kecepatan sudut
        self.angle += self.angular_velocity
        self.angular_velocity += self.angular_acceleration

    def display(self, canvas):
        # Hitung arah rotasi sebagai unit vector
        direction = Vector(math.cos(self.angle), math.sin(self.angle))
        half = direction.multed(self.length / 2)

        # Ujung kiri dan kanan dari pusat
        end1 = self.center.subbed(half)
        end2 = self.center.added(half)

        if self.line is None:
            # Pertama kali buat objek canvas
            self.line = canvas.create_line(end1.x, end1.y, end2.x, end2.y, width=4, fill="blue")
            self.circle1 = canvas.create_oval(end1.x-6, end1.y-6, end1.x+6, end1.y+6, fill="red")
            self.circle2 = canvas.create_oval(end2.x-6, end2.y-6, end2.x+6, end2.y+6, fill="red")
        else:
            # Update posisi objek
            canvas.coords(self.line, end1.x, end1.y, end2.x, end2.y)
            canvas.coords(self.circle1, end1.x-6, end1.y-6, end1.x+6, end1.y+6)
            canvas.coords(self.circle2, end2.x-6, end2.y-6, end2.x+6, end2.y+6)

        return self.angle, self.angular_velocity, self.angular_acceleration


# ==========================
# Fungsi Update Layar
# ==========================
def update():
    baton.update()
    angle, a_vel, a_acc = baton.display(canvas)

    # Tampilkan informasi di layar
    info_text = (
        f"Angle (rad) = {angle:.3f}\n"
        f"Angular Velocity = {a_vel:.3f}\n"
        f"Angular Acceleration = {a_acc:.5f}"
    )
    label_info.config(text=info_text)

    window.after(30, update)  # Loop animasi


# ==========================
# Setup Tkinter Window
# ==========================
WIDTH, HEIGHT = 600, 400
window = tk.Tk()
window.title("Simulasi Rotasi Sudut - The Nature of Code 3.2")

canvas = tk.Canvas(window, width=WIDTH, height=HEIGHT, bg="white")
canvas.pack()

label_info = tk.Label(window, text="", font=("Arial", 12), justify="left")
label_info.pack()

# Buat baton di tengah layar
center = Vector(WIDTH / 2, HEIGHT / 2)
baton = Baton(center, length=150)

# Mulai animasi
update()
window.mainloop()


In [25]:
import tkinter as tk
import math
import random
from vector import Vector  # Import Vector dari vector.py

def constrain(val, min_val, max_val):
    return max(min_val, min(val, max_val))

class Attractor:
    def __init__(self, canvas, x=320, y=120):
        self.canvas = canvas
        self.position = Vector(x, y)
        self.mass = 20
        self.G = 1
        r = self.mass * 2
        self.id = canvas.create_oval(x - r, y - r, x + r, y + r, fill="red")

    def attract(self, mover):
        force = Vector(self.position.x - mover.position.x, self.position.y - mover.position.y)
        distance = force.mag()
        distance = constrain(distance, 5, 25)
        force.normalize()
        strength = (self.G * self.mass * mover.mass) / (distance * distance)
        force.mult(strength)
        return force

    def display(self):
        r = self.mass * 2
        self.canvas.coords(self.id, self.position.x - r, self.position.y - r,
                                  self.position.x + r, self.position.y + r)

class Mover:
    def __init__(self, canvas, x, y, mass):
        self.canvas = canvas
        self.mass = mass
        self.radius = self.mass * 8
        self.position = Vector(x, y)
        self.velocity = Vector(random.uniform(-1, 1), random.uniform(-1, 1))
        self.acceleration = Vector(0, 0)

        self.angle = 0
        self.angle_velocity = 0
        self.angle_acceleration = 0

        r = self.radius
        self.id_circle = canvas.create_oval(x - r, y - r, x + r, y + r, fill="gray", outline="black")
        self.id_line = canvas.create_line(x, y, x + r, y, width=2, fill="black")

    def apply_force(self, force):
        f = force.copy()
        f.div(self.mass)
        self.acceleration.add(f)

    def update(self):
        self.velocity.add(self.acceleration)
        self.position.add(self.velocity)

        self.angle_acceleration = self.acceleration.x / 10.0
        self.angle_velocity += self.angle_acceleration
        self.angle_velocity = constrain(self.angle_velocity, -0.1, 0.1)
        self.angle += self.angle_velocity

        self.acceleration.mult(0)

    def show(self):
        x, y, r = self.position.x, self.position.y, self.radius
        self.canvas.coords(self.id_circle, x - r, y - r, x + r, y + r)

        end_x = x + r * math.cos(self.angle)
        end_y = y + r * math.sin(self.angle)
        self.canvas.coords(self.id_line, x, y, end_x, end_y)

# ========================
# Main Program
# ========================
root = tk.Tk()
root.title("Example 3.2: Forces with Angular Motion - Nature of Code")

canvas = tk.Canvas(root, width=640, height=240, bg="white")
canvas.pack()

attractor = Attractor(canvas)

movers = []
for _ in range(20):
    x = random.uniform(0, 640)
    y = random.uniform(0, 240)
    mass = random.uniform(0.1, 2)
    movers.append(Mover(canvas, x, y, mass))

def animate():
    attractor.display()
    for mover in movers:
        force = attractor.attract(mover)
        mover.apply_force(force)
        mover.update()
        mover.show()
    root.after(30, animate)

animate()
root.mainloop()


In [ ]:
#SCRIPT 3 : # Simulasi Peluru Meriam dengan Rotasi
import tkinter as tk
import math
from vector import Vector  # Pastikan vector.py ada di folder yang sama

class CannonBall:
    def __init__(self, canvas, x, y):
        self.canvas = canvas
        self.position = Vector(x, y)
        self.velocity = Vector(0, 0)
        self.acceleration = Vector(0, 0)
        self.r = 8
        self.topspeed = 10
        self.id = canvas.create_oval(x - self.r, y - self.r, x + self.r, y + self.r, fill="black")

    def apply_force(self, force):
        self.acceleration.add(force)

    def update(self):
        self.velocity.add(self.acceleration)
        self.velocity.limit(self.topspeed)
        self.position.add(self.velocity)
        self.acceleration.mult(0)

    def display(self):
        x, y, r = self.position.x, self.position.y, self.r
        self.canvas.coords(self.id, x - r, y - r, x + r, y + r)

class Cannon:
    def __init__(self, canvas, x, y):
        self.canvas = canvas
        self.position = Vector(x, y)
        self.angle = -math.pi / 4  # -45 derajat
        self.length = 50
        self.id = None

    def draw(self):
        x, y = self.position.x, self.position.y
        end_x = x + self.length * math.cos(self.angle)
        end_y = y + self.length * math.sin(self.angle)
        if self.id is None:
            self.id = self.canvas.create_line(x, y, end_x, end_y, width=10, fill="gray")
        else:
            self.canvas.coords(self.id, x, y, end_x, end_y)

    def rotate(self, delta_angle):
        self.angle += delta_angle
        self.angle = max(-math.pi / 2, min(self.angle, 0))  # constrain sudut

    def shoot_force(self, magnitude=10):
        force = Vector(math.cos(self.angle), math.sin(self.angle))
        force.mult(magnitude)
        return force

# Setup Tkinter
root = tk.Tk()
root.title("Exercise 3.2 Cannon - Nature of Code")

WIDTH, HEIGHT = 640, 360
canvas = tk.Canvas(root, width=WIDTH, height=HEIGHT, bg="white")
canvas.pack()

cannon = Cannon(canvas, 50, 300)
ball = CannonBall(canvas, cannon.position.x, cannon.position.y)
shot = False

def animate():
    global ball, shot
    canvas.delete("background_text")

    cannon.draw()

    if shot:
        gravity = Vector(0, 0.2)
        ball.apply_force(gravity)
        ball.update()

    ball.display()

    # Reset jika jatuh
    if ball.position.y > HEIGHT:
        ball = CannonBall(canvas, cannon.position.x, cannon.position.y)
        shot = False

    # Tampilkan informasi di layar
    angle_deg = math.degrees(cannon.angle)
    canvas.create_text(10, 10, anchor="nw", text=f"Angle: {angle_deg:.1f}°", tag="background_text", font=("Arial", 14))

    pos = ball.position
    vel = ball.velocity
    acc = ball.acceleration
    canvas.create_text(10, 30, anchor="nw", text=f"Pos: ({pos.x:.1f}, {pos.y:.1f})", tag="background_text", font=("Arial", 12))
    canvas.create_text(10, 50, anchor="nw", text=f"Vel: ({vel.x:.2f}, {vel.y:.2f})", tag="background_text", font=("Arial", 12))
    canvas.create_text(10, 70, anchor="nw", text=f"Acc: ({acc.x:.2f}, {acc.y:.2f})", tag="background_text", font=("Arial", 12))

    root.after(30, animate)

def on_key_press(event):
    global shot
    if event.keysym == 'Left':
        cannon.rotate(-0.1)
    elif event.keysym == 'Right':
        cannon.rotate(0.1)
    elif event.keysym == 'space' and not shot:
        shot = True
        force = cannon.shoot_force(10)
        ball.apply_force(force)

root.bind("<KeyPress>", on_key_press)

animate()
root.mainloop()


In [39]:
#SCRIPT 4 : # Simulasi mobil kotak yang bergerak mengikuti arah mouse
import tkinter as tk
import math
from vector import Vector

class Mover:
    def __init__(self, canvas, width, height):
        self.canvas = canvas  # tempat menggambar
        self.width = width    # lebar layar
        self.height = height  # tinggi layar
        self.position = Vector(width / 2, height / 2)  # posisi awal di tengah
        self.velocity = Vector(0, 0)  # kecepatan awal (diam)
        self.acceleration = Vector(0, 0)  # percepatan awal
        self.topspeed = 4  # kecepatan maksimum
        self.r = 16  # ukuran
        self.id = canvas.create_polygon(0, 0, 0, 0, 0, 0, fill="gray", outline="black")  # bentuk kotak

    def update(self, mouse_x, mouse_y):
        mouse = Vector(mouse_x, mouse_y)  # posisi mouse
        dir = mouse.subbed(self.position)    # arah menuju mouse
        dir.normalize()  # buat panjang vektor = 1
        dir.mult(0.5)    # perkecil pengaruhnya
        self.acceleration = dir  # set percepatan
        
        self.velocity.add(self.acceleration)  # tambah kecepatan
        self.velocity.limit(self.topspeed)    # batasi kecepatan maks
        self.position.add(self.velocity)      # update posisi

    def display(self):
        angle = self.velocity.heading()  # sudut berdasarkan arah gerak
        
        w, h = 30, 10  # ukuran kotak
        cx, cy = self.position.x, self.position.y  # posisi tengah
        
        cos_a = math.cos(angle)  # hitung komponen x
        sin_a = math.sin(angle)  # hitung komponen y
        
        points = []
        for dx, dy in [(-w/2, -h/2), (w/2, -h/2), (w/2, h/2), (-w/2, h/2)]:
            x = cx + dx * cos_a - dy * sin_a  # rotasi titik x
            y = cy + dx * sin_a + dy * cos_a  # rotasi titik y
            points.extend((x, y))
        
        self.canvas.coords(self.id, *points)  # update posisi kotak

    def check_edges(self):
        if self.position.x > self.width:  # jika keluar kanan
            self.position.x = 0           # muncul di kiri
        elif self.position.x < 0:         # jika keluar kiri
            self.position.x = self.width   # muncul di kanan
            
        if self.position.y > self.height:  # jika keluar bawah
            self.position.y = 0            # muncul di atas
        elif self.position.y < 0:          # jika keluar atas
            self.position.y = self.height  # muncul di bawah

# Setup Tkinter
root = tk.Tk()
root.title("Example 3.3: Pointing in the Direction of Motion")

WIDTH, HEIGHT = 640, 240
canvas = tk.Canvas(root, width=WIDTH, height=HEIGHT, bg="white")
canvas.pack()

mover = Mover(canvas, WIDTH, HEIGHT)

def animate():
    canvas.delete("text")
    mouse_x = canvas.winfo_pointerx() - canvas.winfo_rootx()  # posisi mouse x
    mouse_y = canvas.winfo_pointery() - canvas.winfo_rooty()  # posisi mouse y

    mover.update(mouse_x, mouse_y)  # update posisi kotak
    mover.check_edges()             # cek tepi layar
    mover.display()                 # gambar ulang
    
    # Tampilkan teks posisi dan kecepatan
    canvas.create_text(10, 10, anchor="nw",
                       text=f"Pos: ({mover.position.x:.1f}, {mover.position.y:.1f})",
                       font=("Arial", 12), tag="text")
    canvas.create_text(10, 30, anchor="nw",
                       text=f"Vel: ({mover.velocity.x:.2f}, {mover.velocity.y:.2f})",
                       font=("Arial", 12), tag="text")

    root.after(30, animate)  # ulangi setiap 30ms

animate()
root.mainloop()


In [ ]:
# SCRIPT 5 : Simulasi Konversi Koordinat Polar ke Kartesian
import tkinter as tk
import math

# === Setup Tkinter ===
root = tk.Tk()
root.title("Example 3.4: Polar to Cartesian")

WIDTH, HEIGHT = 640, 240
canvas = tk.Canvas(root, width=WIDTH, height=HEIGHT, bg="white")
canvas.pack()

# === Inisialisasi variabel polar ===
r = HEIGHT * 0.45     # Jari-jari (radius)
theta = 0             # Sudut awal

# Objek lingkaran dan garis
circle_id = canvas.create_oval(0, 0, 0, 0, fill="gray", outline="black", width=2)
line_id = canvas.create_line(0, 0, 0, 0, fill="black", width=2)

def animate():
    global theta

    canvas.delete("text")
    canvas.configure(bg="white")

    # Konversi polar ke kartesian
    x = r * math.cos(theta)
    y = r * math.sin(theta)

    # Pusat koordinat di tengah canvas
    center_x = WIDTH / 2
    center_y = HEIGHT / 2

    # Koordinat akhir (x, y) ditranslasi ke posisi canvas
    px = center_x + x
    py = center_y + y

    # Update garis dari pusat ke titik
    canvas.coords(line_id, center_x, center_y, px, py)

    # Update lingkaran di posisi (px, py)
    radius = 24
    canvas.coords(circle_id, px - radius, py - radius, px + radius, py + radius)

    # Tampilkan data
    canvas.create_text(10, 10, anchor="nw", text=f"theta = {theta:.2f} rad", font=("Arial", 12), tag="text")
    canvas.create_text(10, 30, anchor="nw", text=f"x = {x:.1f}", font=("Arial", 12), tag="text")
    canvas.create_text(10, 50, anchor="nw", text=f"y = {y:.1f}", font=("Arial", 12), tag="text")

    # Update sudut theta untuk rotasi
    theta += 0.02

    root.after(30, animate)

animate()
root.mainloop()


In [36]:
import tkinter as tk
import math
from vector import Vector

class Vehicle:
    def __init__(self, canvas, x, y):
        self.canvas = canvas
        self.position = Vector(x, y)
        self.velocity = Vector(0, 0)
        self.acceleration = Vector(0, 0)
        self.r = 6
        self.maxspeed = 8
        self.maxforce = 0.2
        self.id = None  # Untuk objek segitiga

    def update(self):
        self.velocity.add(self.acceleration)
        self.velocity.limit(self.maxspeed)
        self.position.add(self.velocity)
        self.acceleration.mult(0)

    def applyForce(self, force):
        self.acceleration.add(force)

    def seek(self, target):
        desired = target.sub(self.position)
        desired.setMag(self.maxspeed)
        steer = desired.sub(self.velocity)
        steer.limit(self.maxforce)
        self.applyForce(steer)

    def show(self):
        x = self.position.x
        y = self.position.y
        angle = self.velocity.heading()

        # Hitung titik segitiga relatif ke posisi dan sudut heading
        r = self.r
        # Titik segitiga pada arah velocity
        p1 = (x + math.cos(angle) * r * 2, y + math.sin(angle) * r * 2)
        # Titik kiri bawah
        p2 = (x + math.cos(angle + 2.5) * r * 2, y + math.sin(angle + 2.5) * r * 2)
        # Titik kanan bawah
        p3 = (x + math.cos(angle - 2.5) * r * 2, y + math.sin(angle - 2.5) * r * 2)

        points = [p1[0], p1[1], p2[0], p2[1], p3[0], p3[1]]

        if self.id is None:
            self.id = self.canvas.create_polygon(points, fill="gray", outline="black")
        else:
            self.canvas.coords(self.id, *points)

# Setup Tkinter
root = tk.Tk()
root.title("NOC 5.01 Seek - Nature of Code")

WIDTH, HEIGHT = 640, 240
canvas = tk.Canvas(root, width=WIDTH, height=HEIGHT, bg="white")
canvas.pack()

vehicle = Vehicle(canvas, WIDTH / 2, HEIGHT / 2)

def animate():
    canvas.delete("mouse_target")
    mouse_pos = Vector(canvas.winfo_pointerx() - root.winfo_rootx(),
                       canvas.winfo_pointery() - root.winfo_rooty())
    # Gambarkan lingkaran di posisi mouse
    r = 24
    canvas.create_oval(mouse_pos.x - r, mouse_pos.y - r,
                       mouse_pos.x + r, mouse_pos.y + r,
                       fill="lightgray", outline="black", width=2, tag="mouse_target")

    # Vehicle seek posisi mouse
    vehicle.seek(mouse_pos)
    vehicle.update()
    vehicle.show()

    root.after(30, animate)

animate()
root.mainloop()


AttributeError: 'NoneType' object has no attribute 'setMag'